<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN04/4.1-Optimierung.ipynb)

angelehnt an:
- [Dive into Deep Learning, Zhang, Aston and Lipton, Zachary C. and Li, Mu and Smola, Alexander J.](https://d2l.ai/index.html)
- [Dive into Deep Learning](https://arxiv.org/abs/2106.11342)

# Optimierung und Deep Learning

In diesem Abschnitt werden wir die Beziehung zwischen Optimierung und Deep Learning sowie die Herausforderungen beim Einsatz von Optimierung beim Deep Learning diskutieren.
- Für ein Deep-Learning-Problem wird in der Regel zunächst eine **Verlustfunktion** (loss $\mathcal{L}$) definiert. 
- Sobald wir die Verlustfunktion kennen, können wir einen **Optimierungsalgorithmus** verwenden, um den Verlust zu minimieren.
- In der Optimierung wird eine Verlustfunktion oft als **Zielfunktion** des Optimierungsproblems bezeichnet. Die meisten Optimierungsalgorithmen befassen sich traditionell mit der **Minimierung**. Wenn wir ein Ziel maximieren müssen, gibt es eine einfache Lösung: Drehen Sie einfach das Vorzeichen des Ziels um.



## Ziel der Optimierung

Obwohl die Optimierung eine Möglichkeit bietet, die Verlustfunktion für Deep Learning zu minimieren, sind die Ziele der *mathematischen Optimierung* und für *Deep Learning* grundlegend verschieden.

- Bei ersterem geht es in erster Linie um die *exakte Minimierung* einer Zielfunktion, während es beim Deep Learning darum geht, ein geeignetes Modell zu finden, das auf einer eine endlichen Menge an Testdaten gut generalisiert.
- Die mathematische Optimierung minimiert eine **exakte Zielfunktion** $f(x)$, die überall innerhalb des Design-Raums ausgewertet werden kann, während die Optimierung das **empirische Risiko** (*empirical risk*) minimiert, welches anhand der Daten geschätzt werden muss.
-  **Trainingsfehler** und **Generalisierungsfehler** sind im Allgemeinen unterschiedlich: Da die Zielfunktion des Optimierungsalgorithmus in der Regel eine Verlustfunktion ist, die auf dem Trainingsdatensatz basiert, besteht das Ziel der Optimierung darin, den Trainingsfehler zu reduzieren.
- Das **Ziel des Deep Learning** (oder allgemeiner gesagt, der *statistischen Inferenz*) ist jedoch die den Generalisierungsfehler zu reduzieren.Um Letzteres zu erreichen, müssen wir beim Deep Learning nebst der Minimierung der Zielfunktion auf den Trainingsdaten auch auf die Reduktion der Überanpassung des Modells achten.


## Risiko und empirisches Risiko

Zur Veranschaulichung der oben genannten unterschiedlichen Ziele, lassen Sie uns das empirische Risiko und das Risiko definieren und visualisieren. 

- Das **empirische Risiko** $g(x)=\bar{\mathcal{L}}$ ist der durchschnittliche Verlust der Zielfunktion auf dem Trainingsdatensatz.
- Das **Risiko** $f(x)=\hat{\mathcal{L}} = \mathbb{E}_{x\sim p(x)} [\mathcal{L}(x)]$ ist der erwartete Verlust für die gesamte Datenpopulation, welche durch die Verteilungsfunktion $p(x)$ definiert ist.

$$\hat{\mathcal{L}} = \mathbb{E}_{x\sim p(x)} [\mathcal{L}(x)]$$

Im Folgenden definieren wir zwei Funktionen:
die Risikofunktion $f$ und die empirische Risikofunktion $g$.

Nehmen wir an, dass wir nur eine endliche Menge an Trainingsdaten haben.
Infolgedessen ist $g$ weniger glatt als $f$.


In [ ]:

# !pip install requests
# !pip install autograd
# !git clone https://github.com/dsgiitr/d2l-pytorch.git
# Homepage
# https://d2l.ai/

# conda install -c conda-forge d2l

%matplotlib inline
import numpy as np
from mpl_toolkits import mplot3d
import torch
import d2l
import matplotlib.pyplot as plt


In [ ]:
def f(x):
    return x * np.cos(np.pi * x)


def g(x):
    return f(x) + 0.2 * np.cos(5 * np.pi * x)


Das nachstehende Diagramm veranschaulicht, dass das Minimum des empirischen Risikos in einem Trainingsdatensatz an einer anderen Stelle liegen kann als das Minimum des Risikos (Generalisierungsfehler).


In [ ]:
d2l.set_figsize((4.5, 2.5))
x = np.arange(0.5, 1.5, 0.01)

plt.figure(figsize=(6,4))
(fig,) = d2l.plt.plot(x, f(x))
(fig,) = d2l.plt.plot(x, g(x))
fig.axes.annotate(
    "empirical risk",
    xy=(1.02, -1.21),
    xytext=(0.5, -1.15),
    arrowprops=dict(arrowstyle="->"),
)
fig.axes.annotate(
    "expected risk",
    xy=(1.1, -1.05),
    xytext=(0.95, -0.5),
    arrowprops=dict(arrowstyle="->"),
)
d2l.plt.xlabel("x")
d2l.plt.ylabel("risk");
plt.grid()


## Herausforderungen für die Optimierung beim Deep Learning

In diesem Abschnitt werden wir uns speziell auf die Leistung von Optimierungsalgorithmen bei der Minimierung der Zielfunktion konzentrieren und nicht auf den Generalisierungsfehler des Modells. 

- Beim Deep Learning sind die meisten Zielfunktionen kompliziert und haben keine analytischen Lösungen. 
- Stattdessen müssen wir *numerische* Optimierungsalgorithmen verwenden. 
- Die Optimierungsalgorithmen die wir für Deep Learning verwenden fallen alle in diese Kategorie.

Bei der Optimierung von Deep Learning gibt es viele Herausforderungen. 
Einige der lästigsten sind 
1. **lokale Minima** (local minima), 
2. **Sattelpunkte** (saddle points) und 
3. **verschwindende Gradienten** (vanishing gradients). 

Werfen wir einen Blick auf darauf.


## 1. Lokale Extrema (Minima)

Für jede beliebige Zielfunktion $f(x)$ gilt: Wenn der Wert von $f(x)$ an der Stelle $x^*$ kleiner ist als die Werte von $f(x)$ an allen anderen Punkten in der Umgebung $U$ von $x \in U(x^*)$, dann könnte $x^*$ ein **lokales Minimum** sein.


Wenn der Wert von $f(x)$ bei $x^*$ das Minimum der Zielfunktion über das gesamte Gebiet ist,
dann ist $f^*=f(x^*)$ das **globale Minimum**.

Ein Beispiel: Für die Funktion

$$f(x) = x \cdot \text{cos}(\pi x) \text{ for } -1.0 \leq x \leq 2.0,$$

können wir das lokale Minimum und das globale Minimum dieser Funktion approximieren.


In [ ]:
x = np.arange(-1.0, 2.0, 0.01)
plt.figure(figsize=(6,4))
(fig,) = d2l.plt.plot(x, f(x))
fig.axes.annotate(
    "local minimum",
    xy=(-0.3, -0.25),
    xytext=(-0.77, -1.0),
    arrowprops=dict(arrowstyle="->"),
)
fig.axes.annotate(
    "global minimum",
    xy=(1.1, -0.95),
    xytext=(0.6, 0.8),
    arrowprops=dict(arrowstyle="->"),
)
d2l.plt.xlabel("x")
d2l.plt.ylabel("f(x)")
plt.grid()


Die Zielfunktion von Deep-Learning-Modellen hat in der Regel **viele lokale Optima**. 
- Wenn sich die numerische Lösung eines Optimierungsproblems in der Nähe des lokalen Optimums befindet, minimiert die durch die letzte Iteration erhaltene numerische Lösung die Zielfunktion möglicherweise nur *lokal* und nicht *global*, da sich der Gradient der Lösungen der Zielfunktion dem Nullpunkt nähert oder diesen erreicht. 
- Nur ein gewisses Mass an Rauschen kann den Parameter aus dem lokalen Minimum herausbringen. Dies ist in der Tat eine der vorteilhaften Eigenschaften des *Minibatch stochastischen Gradientenabstiegs*, bei dem die natürliche Variation der Gradienten über Minibatches in der Lage ist, die Parameter aus den lokalen Minima zu verdrängen.


## 2. Sattelpunkte

Neben lokalen Minima sind Sattelpunkte ein weiterer Grund für das Verschwinden von Gradienten. Ein *Sattelpunkt* ist ein Ort, an dem alle Gradienten einer Funktion verschwinden, der aber weder ein globales noch ein lokales Minimum ist. 
Betrachten wir die Funktion $f(x) = x^3$. Ihre erste und zweite Ableitung verschwinden für $x=0$. Die Optimierung könnte an diesem Punkt zum Stillstand kommen, auch wenn es sich nicht um ein Minimum handelt.

In [ ]:
x = np.arange(-2.0, 2.0, 0.01)
plt.figure(figsize=(6,4))
(fig,) = d2l.plt.plot(x, x**3)
fig.axes.annotate(
    "saddle point", xy=(0, -0.2), xytext=(-0.52, -5.0), arrowprops=dict(arrowstyle="->")
)
d2l.plt.xlabel("x")
d2l.plt.ylabel("f(x)")
plt.grid()


Sattelpunkte in höheren Dimensionen sind sogar noch heimtückischer, wie das folgende Beispiel zeigt. 
- Betrachten wir die Funktion 
$$
f(x, y) = x^2 - y^2
$$.
- Sie hat ihren Sattelpunkt bei $(0, 0)$. Dies ist ein Maximum in Bezug auf $y$ und ein Minimum in Bezug auf $x$. Außerdem *sieht* er aus wie ein Sattel, daher hat diese mathematische Eigenschaft ihren Namen.


In [ ]:
x, y = np.mgrid[-1:1:101j, -1:1:101j]
z = x**2 - y**2

ax = d2l.plt.figure(figsize=(8,5)).add_subplot(111, projection="3d")
ax.plot_wireframe(x, y, z, **{"rstride": 10, "cstride": 10})
ax.plot([0], [0], [0], "rx")
ticks = [-1, 0, 1]
d2l.plt.xticks(ticks)
d2l.plt.yticks(ticks)
ax.set_zticks(ticks)
d2l.plt.xlabel("x")
d2l.plt.ylabel("y")


Wir nehmen an, dass die Eingabe einer Funktion ein $k$-dimensionaler Vektor ist und die
Ausgabe der Funktion ein Skalar ist, so dass ihre hessische Matrix $k$ Eigenwerte hat.

An einer Position, an der der Funktionsgradient verschwindet, könnte ein lokales Minimum, ein lokales Maximum oder ein Sattelpunkt sein!

Die Eigenwerte der Hessematrix

$$\mathcal{H}_{ij}=\frac{\partial^2 f}{\partial x_i \partial x_i}$$

entscheiden über die lokale Krümmung am Ort des stationären Punktes $x^*$.

* Wenn die Eigenwerte der Hessematrix $\mathcal{H}$ der Funktion an der Position des Nullgradienten alle positiv sind, liegt ein lokales Minimum der Funktion vor.
* Wenn die Eigenwerte der Hessematrix $\mathcal{H}$ der Funktion an der Position des Nullgradienten alle negativ sind, liegt ein lokales Maximum der Funktion vor.
* Wenn die Eigenwerte der Hessematrix $\mathcal{H}$ der Funktion an der Null-Gradienten-Position negativ und positiv sind, liegt ein Sattelpunkt für die Funktion vor.

Bei hochdimensionalen Problemen ist die Wahrscheinlichkeit, dass zumindest *einige* der Eigenwerte negativ sind, recht hoch. Dies macht Sattelpunkte wahrscheinlicher als lokale Minima. Wir werden einige Ausnahmen von dieser Situation im nächsten Abschnitt diskutieren, wenn wir die Konvexität einführen. Kurz gesagt, konvexe Funktionen sind solche, bei denen die Eigenwerte der Hessian nie negativ sind. Leider fallen die meisten Deep-Learning-Probleme nicht in diese Kategorie. Nichtsdestotrotz ist es ein großartiges Werkzeug zur Untersuchung von Optimierungsalgorithmen.





## 3. Verschwindende Gradienten

Das wahrscheinlich heimtückischste Problem ist der verschwindende Gradient.
Erinnern Sie sich an unsere häufig verwendeten Aktivierungsfunktionen und ihre Ableitungen.

- Nehmen wir zum Beispiel an, dass wir die Funktion $f(x) = \tanh(x)$ minimieren wollen und wir bei $x = 4$ beginnen. 
- Wie wir sehen können, ist der Gradient von $f$ nahe Null.
- Genauer gesagt ist $f'(x) = 1 - \tanh^2(x)$ und damit $f'(4) = 0,0013$.
- Folglich bleibt die Optimierung lange Zeit stecken, bevor wir Fortschritte machen. Dies ist einer der Gründe dafür, dass das Training von Deep-Learning-Modellen vor der Einführung der ReLU-Aktivierungsfunktion ziemlich schwierig war.

In [ ]:
x = np.arange(-2.0, 5.0, 0.01)
plt.figure(figsize=(6,4))
(fig,) = d2l.plt.plot(x, np.tanh(x))
fig.axes.annotate(
    "vanishing gradient", xy=(4, 1), xytext=(2, 0.0), arrowprops=dict(arrowstyle="->")
)
d2l.plt.xlabel("x")
d2l.plt.ylabel("f(x)");
plt.grid()


Wie wir gesehen haben, ist die Optimierung für Deep Learning mit vielen Herausforderungen verbunden. Glücklicherweise gibt es eine robuste Reihe von Algorithmen, die gut funktionieren und auch für Anfänger einfach zu verwenden sind. Ausserdem ist es nicht unbedingt notwendig, *die* beste Lösung zu finden. Lokale Optima oder sogar Näherungslösungen sind immer noch sehr nützlich.

## Zusammenfassung

* Die Minimierung des Trainingsfehlers ist *keine* Garantie dafür, dass wir den besten Parametersatz zur Minimierung des Generalisierungsfehlers finden.
* Das Optimierungsproblem kann viele lokale Minima haben.
* Das Problem kann noch mehr Sattelpunkte haben, da die Probleme im Allgemeinen nicht konvex sind.
* Verschwindende Gradienten können die Optimierung zum Stillstand bringen. Oft hilft eine Neuparametrisierung des Problems.
* Eine gute Initialisierung der Parameter kann ebenfalls von Vorteil sein.






## Übungen

1. Betrachten Sie ein einfaches MLP mit einem einzigen `hidden layer` mit, sagen wir, $d$ Dimensionen in der versteckten Schicht und einem einzigen Ausgang. Zeigen Sie, dass es für jedes lokale Minimum mindestens $d!$ gleichwertige Lösungen gibt, die sich identisch verhalten.
2. Welche anderen Herausforderungen bei der Optimierung von Deep Learning fallen Ihnen ein?
3. Nehmen Sie an, Sie wollen einen (realen) Ball auf einem (realen) Sattel balancieren.
    A. Warum ist das schwierig?
    B. Kann man diesen Effekt auch für Optimierungsalgorithmen ausnutzen?